In [1]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import scanpy as sc
import anndata
import os
import sys

In [4]:
sys.path.insert(1, '/secure/projects/tacco/')
import tacco as tc

In [5]:
sys.path.insert(1, '/secure/projects/HTAPP_MBC/src/spatial')
from spatial_utils import *

In [4]:
sample_sheet=pd.read_csv(codeDir+"/meta/sampletracking_HTAPP_MBC_spatial.csv")

In [5]:
#These identifiers are set as defaults in many functions. If names are changed, this has to be adapted when calling the functions.
ct_column='cell_type'
sm_methods=["slide_seq","merfish","merfish_bin","exseq","exseq_bin","codex"] #spatial mathod names are hard-coded for now because some methods need special treatments. Can be fixed by giving eatch method it's own variable.
sc_method="scRNAseq"
all_methods=[sc_method,*sm_methods]
all_methods

['scRNAseq',
 'slide_seq',
 'merfish',
 'merfish_bin',
 'exseq',
 'exseq_bin',
 'codex']

In [6]:
run_samples=list(filter(lambda a: not a in [], list(sample_sheet['sample'].unique()))) 
run_samples

['HTAPP-944-SMP-7479',
 'HTAPP-895-SMP-7359',
 'HTAPP-880-SMP-7179',
 'HTAPP-878-SMP-7149',
 'HTAPP-812-SMP-8239',
 'HTAPP-514-SMP-6760',
 'HTAPP-982-SMP-7629',
 'HTAPP-997-SMP-7789',
 'HTAPP-917-SMP-4531',
 'HTAPP-853-SMP-4381',
 'HTAPP-313-SMP-932',
 'HTAPP-330-SMP-1082',
 'HTAPP-783-SMP-4081',
 'HTAPP-364-SMP-1321',
 'HTAPP-213-SMP-6752']

### Annotate

In [8]:
for sample in run_samples:
    print("-------------------"+sample+"-------------------")
    out_dir=out_dir_base+sample 
    adatas=get_and_process_data(sample,data_dir,out_dir,methods=all_methods,process=False)
    sp_typing=run_typing(adatas,out_dir,sample,RCTD=True)
    add_typing_results(adatas,sp_typing)
    update_processed_adatas(adatas,sample,out_dir,sm_methods,overwrite=True,process=False)

-------------------HTAPP-944-SMP-7479-------------------
reading  scRNAseq  from buffer
reading  slide_seq  from buffer
reading  merfish  from buffer
reading  merfish_bin  from buffer
No data for exseq
No data for exseq_bin
No data for codex
++slide_seq++
reading  OT  from buffer
reading  OT_max  from buffer
reading  RCTD  from buffer
++merfish++
reading  OT  from buffer
reading  OT_max  from buffer
reading  RCTD  from buffer
++merfish++
reading  OT  from buffer
reading  OT_max  from buffer
reading  RCTD  from buffer
++merfish++
reading  OT  from buffer
reading  OT_max  from buffer
reading  RCTD  from buffer
++merfish_bin++
reading  OT  from buffer
reading  OT_max  from buffer
reading  RCTD  from buffer
++merfish_bin++
reading  OT  from buffer
reading  OT_max  from buffer
reading  RCTD  from buffer
++merfish_bin++
reading  OT  from buffer
reading  OT_max  from buffer
reading  RCTD  from buffer
Writing slide_seq
Writing merfish
Writing merfish_bin
exseq not available.
exseq_bin not avai

### Group annotations by compartment

In [11]:
for sample in run_samples:
    print(sample)
    out_dir=out_dir_base+sample 
    adatas=get_and_process_data(sample,data_dir,out_dir,methods=all_methods,process=False)
    for method in adatas.index[adatas.index != sc_method]:
        tc.utils.merge_annotation(adatas[method],mapping=compartments,result_key='OT_compartment',annotation_key='OT')
        tc.utils.merge_annotation(adatas[method],mapping=compartments,result_key='OT_max_compartment',annotation_key='OT_max')
        tc.utils.merge_annotation(adatas[method],mapping=compartments,result_key='RCTD_compartment',annotation_key='RCTD')
    update_processed_adatas(adatas,sample,out_dir,sm_methods,overwrite=True,process=False)

HTAPP-944-SMP-7479
reading  scRNAseq  from buffer
reading  slide_seq  from buffer
reading  merfish  from buffer
reading  merfish_bin  from buffer
No data for exseq
No data for exseq_bin
No data for codex
Writing slide_seq
Writing merfish
Writing merfish_bin
exseq not available.
exseq_bin not available.
codex not available.
HTAPP-895-SMP-7359
reading  scRNAseq  from buffer
reading  slide_seq  from buffer
No data for merfish
No data for merfish_bin
reading  exseq  from buffer
No data for exseq_bin
No data for codex
Writing slide_seq
merfish not available.
merfish_bin not available.
Writing exseq
exseq_bin not available.
codex not available.
HTAPP-880-SMP-7179
reading  scRNAseq  from buffer
reading  slide_seq  from buffer
reading  merfish  from buffer
reading  merfish_bin  from buffer
reading  exseq  from buffer
reading  exseq_bin  from buffer
reading  codex  from buffer
Writing slide_seq
Writing merfish
Writing merfish_bin
Writing exseq
Writing exseq_bin
Writing codex
HTAPP-878-SMP-7149


### Add annotations to obs

In [21]:
for sample in run_samples:
    print(sample)
    out_dir=out_dir_base+sample 
    adatas=get_and_process_data(sample,data_dir,out_dir,methods=all_methods,process=False)
    for method in adatas.index[adatas.index != sc_method]:
        for todel in ['RCTD','OT','OT_max']: #delete old entries before adding new ones
            try: 
                del(adatas[method].obs[todel])
            except:
                pass
        
        adatas[method].obsm['RCTD'].fillna(0,inplace=True)
        adatas[method].obsm['OT'].fillna(0,inplace=True)
        adatas[method].obsm['OT_max'].fillna(0,inplace=True)
        typing_to_obs(adatas[method],['RCTD','OT','OT_max'])
    update_processed_adatas(adatas,sample,out_dir,sm_methods,overwrite=True,process=False)

HTAPP-944-SMP-7479
reading  scRNAseq  from buffer
reading  slide_seq  from buffer
reading  merfish  from buffer
reading  merfish_bin  from buffer
No data for exseq
No data for exseq_bin
No data for codex


... storing 'RCTD' as categorical
... storing 'OT' as categorical
... storing 'OT_max' as categorical


Writing slide_seq


... storing 'RCTD' as categorical
... storing 'OT' as categorical
... storing 'OT_max' as categorical


Writing merfish


...

Writing codex
HTAPP-213-SMP-6752
reading  scRNAseq  from buffer
reading  slide_seq  from buffer
No data for merfish
No data for merfish_bin
No data for exseq
No data for exseq_bin
reading  codex  from buffer


... storing 'RCTD' as categorical
... storing 'OT' as categorical
... storing 'OT_max' as categorical


Writing slide_seq


... storing 'RCTD' as categorical
... storing 'OT' as categorical
... storing 'OT_max' as categorical


merfish not available.
merfish_bin not available.
exseq not available.
exseq_bin not available.
Writing codex
